## Quantum Key Distribution

Asja wants to send messages to Balvis who will go abroad but they want to keep touch. But Eve, their curious friend, want to learn what they will talk. Asja and Balvis found a solution called one time padding. Before he leave they randomly produce (for examle by flipping a coin) a binary string with length $n$. Whenever Asja wants to send a message, first she $XOR$ her message with this string and after Belvis has message, he $XOR$ again with the same string. In this situation Eve can't find the message because there are many possible ($2^n$) strings Asja and Balvis can produce by flipping coin.
But Asja and Balvis can produce only finite length string so they have to use same keys over and over. Eve can gain some information using some statictical methods after some long time. So this procedure will not safe after sending $n$ bits.

Quantum Key Distribution solves this problem. We can use quantum computers to generate new keys even Asja and Balvis far from each other.

There are several protocols. We will see BB84.

## Quantum No Clonning Theorem

No-cloning theorem states that it is impossible to create an identical copy of an arbitrary unknown quantum state. Which states that Eve can't make copies of qubits and make measurements after protocol ends.



Let apply two successive Hadamard gates, and measure. 

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
import numpy as np
import random

qreg = QuantumRegister(1)
creg = ClassicalRegister(1)
circ = QuantumCircuit(qreg,creg)

circ.h(qreg[0])

circ.h(qreg[0])

circ.measure(qreg, creg)


In [ ]:
circ.draw(output = 'mpl')


In [ ]:
job = execute(circ,Aer.get_backend('qasm_simulator'), shots=1024)
counts1 = job.result().get_counts(circ)
print(counts1)

We observe $0$ all time. Let's make a measurement between two Hadamard gates also.

In [ ]:
qreg = QuantumRegister(1)
creg = ClassicalRegister(1)
circ = QuantumCircuit(qreg,creg)

circ.h(qreg[0])

circ.measure(qreg, creg)

circ.h(qreg[0])

circ.measure(qreg, creg)


In [ ]:
circ.draw(output = 'mpl')


In [ ]:
job = execute(circ,Aer.get_backend('qasm_simulator'), shots=1024)
counts1 = job.result().get_counts(circ)
print(counts1)

We see $0$ and $1$ randomly. Eve's measurements can break quantum states.

## Bell basis

Our computational basis is $ \lvert 0 \rangle $ and $ \lvert 1 \rangle $ . If we apply Hadamard gate we have  $ \frac{ \lvert 0 \rangle + \lvert 1 \rangle }{\sqrt{2}}$ and  $ \frac{ \lvert 0 \rangle - \lvert 1 \rangle }{\sqrt{2}}$ which are orthogonal too. If we think $ \lvert 0 \rangle $ and $ \lvert 1 \rangle $ as $\uparrow$ and $\rightarrow$, then Bell basis is $\nearrow$ and $\searrow$

## BB48 Protocol

Let Asja wants to send $n$ classical bits to Balvis, but they need a key to secure their message. Asja prepares $4n$ qubits by flipping coins. If she sees a tail, qubit remain zero otherwise she sets qubit to one. Then she flips coin again to decide actually what basis she use for every qubit. So Asja have randomly created $0$ or $1$ qubits in Bell or computational basis. Then she sends qubits to Balvis without sending information about initial values and basis. When he recive qubits, he randomly chooses basis and measures them. After measurement Asja and Balvis shares their basis information using classical channels. Approximately Asja and Balvis aggree on half, $2n$, of them. Focus these $2n$ qubits (Asja and Balvis restart all procudure if there are less then $2n$ aggrements). If they make measurenments on same basis, they should have same values on these qubits. But Eve can listen their coumminication. If Eve listens, there is probility greater than zero that Asja and Balvis has different qubits. They share same random $n$ of $2n$ qubits values and then compair with each other, if there is a different qubit then they restart because Eve is listening. If all qubits are same then probabilty that Eve listen them is too low. They use remaning $n$ qubits' valuses as a key.

This protocol creates a new key when Asja and Balvis want, so Eve never be able to eavesdropp.

* Create quantum circuit with $4*n$ qubits.
* Create a random string or array to decide inital values
* Create a random string or array to decide which basis Asja use
* Create a variable to control wheter Eve is eavesdropping or not, if she is:
 * Create a random string or array to decide basis
 * Make a measurenment
 * Apply same string or array before sending to Balvis
* Create a random string or array to decide which basis Balvis use
* Make a measurenment
* Count qubits which Asja and Balvis use same basis, if it is less than $2*n$ go to step 1
* Take random $n$ of these $2n$ qubits and compait their values. If there is a different one, go to step 1
* Remaining $n$ qubits' values is our key.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
import numpy as np
import random

# Length of message
n = 10
# Number of qubits for protocol
sz = 4*n

qreg = QuantumRegister(sz)
creg = ClassicalRegister(sz)
circ = QuantumCircuit(qreg,creg)

# flip coin for initial values
arr1 = np.random.randint(2, size=sz)
print("Asja's initilazion")
print(arr1)

# apply x gate for 1s, do nothing for 0s
for i in range(sz):
    if arr1[i] == 1:
        circ.x(qreg[sz-i-1])
        
# flip coins for Hadamard gates        
arr2 = np.random.randint(2, size=sz)
print("Asja's Gates")
print(arr2)
for i in range(sz):
    if arr2[i] == 1:
        circ.h(qreg[sz-i-1])
        
# Eve decides randomly to listen or nor        
spy = np.random.randint(2)
if spy == 1:
    # becuse Eve doesn't know which base Asja use she randomly decide that
    arr3 = np.random.randint(2, size=sz)
    print("Eve listens")
    print("Eve's Gates")
    print(arr3)
    for i in range(sz):
        if arr3[i] == 1:
            circ.h(qreg[sz-i-1])
    # measure all qubits
    circ.measure(qreg, creg)
    # prepare qubits same basis as she measure
    for i in range(sz):
        if arr3[i] == 1:
            circ.h(qreg[sz-i-1])

# Belvis measures with random bases    
arr4 = np.random.randint(2, size=sz)
print("Belvis' Gates")
print(arr4)
for i in range(sz):
    if arr4[i] == 1:
        circ.h(qreg[sz-i-1])

# Belvis measures
circ.measure(qreg, creg)

In [ ]:
# circ.draw(output = 'mpl')

In [ ]:
job = execute(circ,Aer.get_backend('qasm_simulator'), shots=1)
counts1 = job.result().get_counts(circ)
print(counts1)

In [ ]:
# because we measure just one time, we have only one possible outcome so this loop iterates only once
for result in counts1.keys():
    # how many times Asja and Balvis agree on bases
    same = 0
    # index of qubits that they agree on
    indx = []
    for i in range(sz):
        if arr2[i] == arr4[i]:
            same = same + 1
            indx.append(i)
    if same < 2*n:
        print("Insufficient qubits, restart the process")
        break
    # shuffle to get random 2n qubits and n control qubits
    random.shuffle(indx)
    # let first n qubits are control bits(because we shuffle whole array first n is also random)
    cont = indx[0:n]
    # boolean flag to stop process if Eve is listening
    b = False
    for i in cont:
        if int(result[i]) != arr1[i]:
            print("Eve listens us")
            b = True
            break
    # Eve is listining so stop the process
    if b:
        break
    # We can reach here if we decide Eve is not listening, check if it is true.
    if spy == 1:
        print("Eve successfuly listens us")
        break
    print("Key is : " + str([result[j] for j in indx[n:(2*n)]]))
    break

## Calculating Probabilty of Asja and Balvis don't Notice Eve

Let Eve listens Asja and Balvis. She makes measurements in random basis for every qubit. Because every qubit is independent, lets think of one qubit. We will calculate the probabilty of Balvis measures correct result as Asja send after Eve's observation. Eve measures a qubit with random basis, and initial new qubit with same basis and value as she measured and send to Balvis. If she choose correct basis, Balvis don't notice Eve because she correctly reproduce same state as Asja. But if Eve choose wrong basis there are two situations: Asja choose $+$ and Eve choose $x$ and vice versa. First case, Eve has $ \lvert 0 \rangle $ or $ \lvert 1 \rangle $ with $\frac{1}{2}$ probabilty, and she sends qubit after applying Hadamard to Balvis and whatever Eve measure again Balvis has $ \lvert 0 \rangle $ or $ \lvert 1 \rangle $ with equal probability.

In other case, Asja prepares qubit in $x$ basis and Eve measures in $+$ basis, again she randomly measure $ \lvert 0 \rangle $ or $ \lvert 1 \rangle $ and send them to Balvis in $+$ basis. Balvis make measurenments in $x$ basis and for both situations he gets random results.

So probabilty Asja and Balvis make different measurenments if Eve involves is $\frac{1}{2}*\frac{1}{2}=\frac{1}{4}$, probabilty Eve choose incorrect basis times probabilty Balvis' measurenment is different then Asja's initial bit. 
So probabilty Asja and Balvis don't notice Eve if she listens them and test $n$ of $2n$ bits they agree on is $1-(\frac{3}{4})^n$ which is greater than $0.999999999$ after $n>72$